In [1]:
import pandas as pd
import numpy as np
import sys, os

from matplotlib import pyplot as plt
sys.path.insert(1, os.path.join(sys.path[0], '../../'))  # for importing local packages from src
sys.path.insert(1, os.path.join(sys.path[0], '../'))  # for importing model config

%reload_ext autoreload
%autoreload 2 

## Datasets

In [2]:
from src.model.rc_dataset import ExperimentDataset


DATASET_NEMA_H0 = '../datasets/dataset_NEMA_NEMA_H0.csv'
DATASET_NEMA_H3 = '../datasets/dataset_NEMA_NEMA_H3.csv'
DATASET_NEMA_H15 = '../datasets/dataset_NEMA_NEMA_H15.csv'

dataset_nema_h0 = ExperimentDataset(csv_path=DATASET_NEMA_H0)
dataset_nema_h3 = ExperimentDataset(csv_path=DATASET_NEMA_H3)
dataset_nema_h15 = ExperimentDataset(csv_path=DATASET_NEMA_H15)

datasets = [
  dataset_nema_h0, 
  dataset_nema_h3,
  dataset_nema_h15
]

## Model configuration

In [35]:
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.model_selection import LeaveOneGroupOut
from src.learning.scorers import nmse_scorer
from pipeline_cnwheat import GroupGenerator, TimeGenerator
from pipeline_base import TrainTestSplitter, DirectTransform, WarmupTransform

# constants
DATASET_ID = 'NEMA_combined'
STATE_SIZE = 7
N_STATE_SAMPLES = 16
state_ids = None

# Reservoir generation
np.random.seed(42)
# NOTE: all measurable reservoirs considered have the same reservoir size of 10.
generate_state_sample = lambda : np.random.choice(10, size=STATE_SIZE, replace=False)


# Readout model
readout_model = Pipeline([
  ('ridge_regression', Ridge(alpha=1, fit_intercept=True))
])
model_param_grid = [{
  'ridge_regression__alpha': 10 ** np.linspace(np.log10(1e-4), np.log10(1e2), 50)
}]

# Regression task pipeline
shared_pipeline_params = {
  # Data generation
  'datasets': datasets,
  'groups': GroupGenerator(day_length=24),
  'time': TimeGenerator(day_length=24),
  
  # Model training and validation
  'readout_model': readout_model,
  'model_param_grid': model_param_grid,
  'model_scorer': nmse_scorer,
  'folds': LeaveOneGroupOut(),
  'train_test_split': TrainTestSplitter(block_size=4, test_ratio=0.5),
}

## Regression Pipelines

In [43]:
from pipeline_base import (
  RCPipeline, 
  Rescale, 
  DaylightMask
)

from pipeline_cnwheat import (
  TargetGenerator, 
  SingleReservoirGenerator,
  MultiReservoirGenerator,
  TargetReservoirGenerator,
  GroupRescale
)

from model_config_cnwheat import (
  baseline_reservoirs,
  heterogeneous_reservoirs, 
  final_targets, 
  measurable_reservoirs,
)

### Input and Physiological Tasks

In [46]:
target_state_pairs = [(target, state_var) for target in final_targets for state_var in measurable_reservoirs]
target_env_pairs = [(name, target, env_targets) for target in final_targets for (name, env_targets) in baseline_reservoirs]
target_het_pairs = [(name, target, state_vars) for target in final_targets for (name, state_vars) in heterogeneous_reservoirs]

print(final_targets)
print(measurable_reservoirs)
print(baseline_reservoirs)
print(heterogeneous_reservoirs)

['input_air_temperature', 'input_humidity', 'input_PARi', 'output__axes__Total_Transpiration', 'output__axes__sum_respi_roots', 'output__axes__sum_respi_shoot', 'output__custom__PARa']
['state__An', 'state__Ts', 'state__gs', 'state__Tr']
[('env_all', ['input_air_temperature', 'input_humidity', 'input_PARi']), ('env_temp', ['input_air_temperature']), ('env_humidity', ['input_humidity']), ('env_PAR', ['input_PARi'])]
[('state__het_all', ('state__An', 'state__Transpiration', 'state__Ts', 'state__gs', 'state__Ag', 'state__Tr', 'state__Rd', 'state__sum_respi', 'state__PARa'))]


In [38]:
input_phys_transforms = [
  WarmupTransform(warmup_days=4, day_length=24),
  DirectTransform()
]


In [52]:
def generate_single(target_state_pairs, state_ids):
  for target, state_var in target_state_pairs:
    pipeline = RCPipeline(
      metadata={'target_id': target, 'reservoir_id': state_var, 'dataset_id': DATASET_ID},
      # Data generation
      target=TargetGenerator(target=target),
      reservoir=SingleReservoirGenerator(state_var=state_var, state_ids=state_ids),
      # Data transformation
      transforms=input_phys_transforms,
      # Data preprocessing
      preprocessing=[
        DaylightMask(day_length=24, start=5, end=21),
        Rescale(per_feature=False)
      ],
      **shared_pipeline_params,
    )
    yield pipeline


def generate_multi(target_het_pairs, state_ids):
  for name, target, state_vars in target_het_pairs:
    pipeline = RCPipeline(
      metadata={'target_id': target, 'reservoir_id': name, 'dataset_id': DATASET_ID},
      # Data generation
      target=TargetGenerator(target=target),
      reservoir=MultiReservoirGenerator(state_vars=state_vars, state_ids=state_ids),
      # Data transformation
      transforms=input_phys_transforms,
      # Data preprocessing
      preprocessing=[
        DaylightMask(day_length=24, start=5, end=21),
        GroupRescale(datasets=datasets, state_vars=state_vars)
      ],
      **shared_pipeline_params,
    )
    yield pipeline


def generate_env(target_env_pairs):
  for name, target, env_targets in target_env_pairs:
    pipeline = RCPipeline(
      metadata={'target_id': target, 'reservoir_id': name, 'dataset_id': DATASET_ID},
      # Data generation
      target=TargetGenerator(target=target),
      reservoir=TargetReservoirGenerator(targets=env_targets),
       # Data transformation
      transforms=input_phys_transforms,
      # Data preprocessing
      preprocessing=[
        DaylightMask(day_length=24, start=5, end=21),
        Rescale(per_feature=True)
      ],
      **shared_pipeline_params,
    )
    yield pipeline

In [54]:
from itertools import chain


def generate_pipelines(state_ids):
  pipelines = list(chain(
    generate_single(target_state_pairs, state_ids),
    generate_multi(target_het_pairs, state_ids),
    generate_env(target_env_pairs)
  ))
  return pipelines


np.random.seed(42)
state_ids = generate_state_sample()
pipelines = generate_pipelines(state_ids)
print(f'Total pipelines: {len(pipelines)}')
print(f'Total runs: {len(pipelines) * N_STATE_SAMPLES}')

Total pipelines: 63
Total runs: 1008


In [56]:
pipes = generate_pipelines(state_ids)
n_pipelines = len(pipes)
print(n_pipelines)

63


In [57]:
from tqdm import tqdm
from itertools import chain
from pipeline_base import execute_pipeline


RESULTS_FILE = 'results_cnwheat_input_phys_16sample.csv'

np.random.seed(42)
all_state_samples = [generate_state_sample() for _ in range(N_STATE_SAMPLES)]
total_loops = n_pipelines * N_STATE_SAMPLES
results = []


with tqdm(total=total_loops) as pbar:
  for i_state_sample, state_ids in enumerate(all_state_samples):
    pipelines = generate_pipelines(state_ids)
    for pipeline in pipelines:
      result = execute_pipeline(pipeline)
      result['state_sample'] = i_state_sample
      results.append(result)
      pbar.update(1)


results_df = pd.DataFrame.from_dict(results)
results_df.to_csv(RESULTS_FILE)
print(f'Saved scores to {RESULTS_FILE}')

  3%|▎         | 28/1008 [00:11<06:20,  2.57it/s]f:\thesis-cyborg-plants\notebooks\regression_tests\..\pipeline_cnwheat.py:217: RuntimeWarning: Mean of empty slice.
  return X, y, groups, time
C:\Users\Max\anaconda3\envs\rc-plants\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\Max\anaconda3\envs\rc-plants\lib\site-packages\numpy\core\_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\Max\anaconda3\envs\rc-plants\lib\site-packages\numpy\core\_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
C:\Users\Max\anaconda3\envs\rc-plants\lib\site-packages\numpy\core\_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
  3%|▎         | 29/1008 [00:12<06:52,

Saved scores to results_cnwheat_input_phys_16sample.csv
